In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import talib as ta
from datetime import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()
from sklearn.metrics import accuracy_score, r2_score
from datetime import datetime as dt, timedelta

In [2]:
def get_prices(ticker, start_date, end_date):
    start = datetime.strptime(start_date, '%d/%m/%Y')
    end = datetime.strptime(end_date, '%d/%m/%Y')
    f = web.DataReader(ticker, 'morningstar', start, end)
    df = pd.DataFrame(f)
    
    return df

## SQL Connection

In [3]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

def create_connection():
    # Login details: 
    DSN_Name = 'traderDSN' ; Login_ID = 'basic' ; pwd = 'pwd1'

    # The engine does the same job as a connection and a cursor
    engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 

    return engine 

# Build query here:
query = ''' 
SELECT * FROM {schema}.{table_name}

'''.format(schema='stocks', table_name='Adjusted_Fortune_500')
print('Query:', query)
print('='*60)


engine = create_connection()

# Importing a Pandas Dataframe: 
df1 = pd.read_sql(query,engine)

df1.head()

Query:  
SELECT * FROM stocks.Adjusted_Fortune_500




,index,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,date,Name,Ticker
0,0,84.1400,84.5400,83.8200,84.5100,84.5100,10713391,0.0000,1.0000,2018-05-21,Walmart,WMT
1,0,84.2400,84.4400,83.4900,83.6400,83.6400,12043137,0.0000,1.0000,2018-05-18,Walmart,WMT
2,0,87.0400,87.5900,83.8500,84.4900,84.4900,29720738,0.0000,1.0000,2018-05-17,Walmart,WMT
3,0,84.8700,86.6250,84.7400,86.1300,86.1300,13733627,0.0000,1.0000,2018-05-16,Walmart,WMT
4,0,84.0600,84.8800,83.8000,84.5200,84.5200,9322923,0.0000,1.0000,2018-05-15,Walmart,WMT


In [4]:
prices = df1.copy()

prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
prices.head()

,open,high,low,close,volume,date,name,symbol
0,84.1400,84.5400,83.8200,84.5100,10713391,2018-05-21,Walmart,WMT
1,84.2400,84.4400,83.4900,83.6400,12043137,2018-05-18,Walmart,WMT
2,87.0400,87.5900,83.8500,84.4900,29720738,2018-05-17,Walmart,WMT
3,84.8700,86.6250,84.7400,86.1300,13733627,2018-05-16,Walmart,WMT
4,84.0600,84.8800,83.8000,84.5200,9322923,2018-05-15,Walmart,WMT


In [5]:
    %%time 
    
    # Create engine and query 
    engine = create_connection()
    query = ''' 
    SELECT * FROM {schema}.{table_name}
    '''.format(schema='stocks', table_name='Adjusted_Fortune_500')

    # Importing a Pandas Dataframe
    prices = pd.read_sql(query,engine)
    prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
    prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})
    
    # Reformat column types
    prices['open']  = prices['open'].astype(float)
    prices['high']  = prices['high'].astype(float)
    prices['low']   = prices['low'].astype(float)
    prices['close'] = prices['close'].astype(float)
    prices['volume']= prices['volume'].astype(float)

    prices['date'] = pd.to_datetime(prices['date'])

    prices.set_index(['name', 'symbol', 'date'], inplace=True)
    prices = prices.sort_index(level=1, ascending=True)

Wall time: 24.6 s


In [6]:
prices.head()

open     high      low    close      volume
name  symbol date                                                    
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0

In [7]:
    # Calculated columns
    prices  = prices.copy().loc[(prices.loc[:,'open':'volume'] != 0 ).all(axis=1)] # Don't touch this (:
    
    # Group over the symbols. 
    prices_g = prices.groupby(level=['name'])
    
    EMA_lengths = [20, 50, 200]  # Calculate many moving averages, of closing prices
    for l in EMA_lengths: 
        prices['EMA_'+str(l)] = prices_g.close.apply(lambda x: ta.EMA(x,l))   

    prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
    prices['RSI'] = prices_g.close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

    def CCI_func(x):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.CCI(high=xh, low=xl, close=xc, timeperiod=5)
   
    prices['CCI'] = prices_g.apply(CCI_func).reset_index(level=0,).drop(columns='name')    # created an extra column. 

    prices['Momentum'] = prices_g.close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum changed to prices_g.close

    def Stoch_fast(x, index):
        xh = x.high 
        xl = x.low 
        xc = x.close 
        return ta.STOCHF(high=xh, low=xl, close=xc, fastk_period=14, fastd_period=3)[index] # Params 14, 3    
    
    #prices['Stoch_fastk'] = prices_g.apply(Stoch_fast, 0).reset_index(level=1,).drop(columns='symbol')   # Stochastic fast indicator
    #prices['Stoch_fastd'] = prices_g.apply(Stoch_fast, 1).reset_index(level=1,).drop(columns='symbol')

   # def OBV_func(x):
    #    xc = x.close
   #     xv = x.volume
    #    return ta.OBV(xc, xv) 
    #prices['OBV'] = prices_g.apply(OBV_func).reset_index(level=1,).drop(columns='symbol')  # On Balance Volume
    

    def bband_func(x, index):
        xc = x.close
        return ta.BBANDS(xc)[index]
    
    prices['bbandupper'] = prices_g.apply(bband_func, 0).reset_index(level=0,).drop(columns='name')
    prices['bbandmiddle'] = prices_g.apply(bband_func, 1).reset_index(level=0,).drop(columns='name') 
    prices['bbandlower'] =  prices_g.apply(bband_func, 2).reset_index(level=0,).drop(columns='name')


In [8]:
prices['bbandlowerii'] =  prices['bbandlower']/prices['close']
prices['bbandupperii'] =  prices['bbandupper']/prices['close']

In [9]:
prices.head(205)

open     high      low    close      volume  \
name  symbol date                                                       
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0   
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0   
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0   
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0   
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0   
             2016-11-08  24.30  26.5900  24.3000  26.4000   6469979.0   
             2016-11-09  26.83  29.1400  26.0000  28.7200  10675856.0   
             2016-11-10  29.08  29.7000  28.3000  29.1000   8189887.0   
             2016-11-11  29.18  29.9900  27.6300  29.3000   5337239.0   
             2016-11-14  29.50  29.9400  28.6100  29.6200   5156737.0   
             2016-11-15  28.94  30.2300  28.8300  30.2100   3668883.0   
             2016-11-16  29.69  31.6900  29.6600  31.4200   6150098.0   
             2016-11-17  31.30  32.0200  30.7700  31.8500   4693203.0   
             2016-11-18  31.73  31.7700  30.1600  30.4200   4421730.0   
             2016-11-21  30.89  30.9800  29.5010  30.6900   2986107.0   
             2016-11-22  31.10  31.9100  30.5400  31.3400   4182721.0   
             2016-11-23  31.00  31.9300  30.0000  31.5100   4112265.0   
             2016-11-25  31.24  32.1900  31.1100  31.7000   3109628.0   
             2016-11-28  31.48  31.8000  30.7000  30.8600   5121935.0   
             2016-11-29  30.02  30.4999  29.7300  29.7300   4927708.0   
             2016-11-30  30.06  30.4400  28.8500  28.9700   5131539.0   
             2016-12-01  28.84  29.4200  28.6000  28.8800   5006186.0   
             2016-12-02  28.86  29.6250  28.7300  29.0400   3870676.0   
             2016-12-05  29.03  31.2768  28.8900  31.2200   7034136.0   
             2016-12-06  30.71  31.1900  30.2100  31.1500   3248595.0   
             2016-12-07  31.30  31.8900  30.7653  30.9000   5778259.0   
             2016-12-08  31.28  31.8400  30.8000  31.3100   3064978.0   
             2016-12-09  31.31  32.1100  31.0400  32.0500   4344295.0   
             2016-12-12  32.16  32.3500  30.8500  31.2400   3416012.0   
             2016-12-13  31.40  31.7300  30.0300  30.2400   4215143.0   
...                        ...      ...      ...      ...         ...   
             2017-07-14  36.20  36.4400  35.9800  36.3200   3966487.0   
             2017-07-17  36.44  36.6900  36.2050  36.4000   3453010.0   
             2017-07-18  36.40  36.4800  35.8000  36.4300   3049846.0   
             2017-07-19  36.61  36.6200  36.0700  36.5000   4431844.0   
             2017-07-20  35.95  37.3100  35.9500  36.7200   4745762.0   
             2017-07-21  36.55  36.8590  36.5060  36.6200   2449086.0   
             2017-07-24  36.59  36.8800  36.5100  36.6900   2244773.0   
             2017-07-25  37.07  37.3800  36.8650  37.3000   3403471.0   
             2017-07-26  37.42  37.6100  36.5600  36.6300   3390179.0   
             2017-07-27  36.84  37.0700  36.0850  36.3600   2044059.0   
             2017-07-28  36.17  36.3200  35.5400  35.6400   2851126.0   
             2017-07-31  35.97  36.4400  35.6650  36.4000   2137320.0   
             2017-08-01  36.46  36.7400  35.9100  36.3800   1918688.0   
             2017-08-02  36.27  37.3300  36.2000  37.2800   2962758.0   
             2017-08-03  37.28  37.6000  36.9400  37.0900   2884558.0   
             2017-08-04  37.34  37.5000  36.7300  37.4200   2285810.0   
             2017-08-07  38.00  38.4700  37.6100  37.9900   4914994.0   
             2017-08-08  38.50  40.3620  38.3600  39.5800   8674921.0   
             2017-08-09  39.21  39.4500  38.4000  38.6200   3692522.0   
             2017-08-10  38.12  38.7800  37.7150  37.7800   3515585.0   
             2017-08-11  37.21  37.9100  36.9300  37.7800   2936153.0   
             2017-08-14  38.11  38.1580  37.1250  37.1900   2129458.0   
           

In [10]:
#US_yields=pd.read_pickle(r'C:\Users\Student\Documents\Trading\US_yieldcurve.pkl')

In [11]:
import quandl
US_yields=quandl.get("USTREASURY/YIELD", authtoken="wdLjKc5NXiuwUhFLByKm")
US_yields=US_yields[['3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '30 YR']]

In [12]:
US_yields.index.rename('date', inplace=True)

In [13]:
prices.reset_index(level='date',inplace=True)

In [14]:
test=pd.merge(prices, US_yields, how='left', left_on='date' , right_index=True)

In [15]:
test.set_index('date',append=True,inplace=True)

In [16]:
test.sort_index(level=1, ascending=True,inplace=True)

In [17]:
test.head()

open     high      low    close      volume  EMA_20  \
name  symbol date                                                               
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0     NaN   
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0     NaN   
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0     NaN   
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0     NaN   
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0     NaN   

                         EMA_50  EMA_200  EMA_ratio  RSI  ...    bbandupperii  \
name  symbol date                                         ...                   
Alcoa AA     2016-11-01     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-02     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-03     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-04     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-07     NaN      NaN        NaN  NaN  ...        1.037733   

                         3 MO  6 MO  1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  \
name  symbol date                                                          
Alcoa AA     2016-11-01  0.35  0.50  0.65  0.83  0.99  1.30  1.61   1.83   
             2016-11-02  0.37  0.51  0.64  0.81  0.98  1.26  1.57   1.81   
             2016-11-03  0.38  0.52  0.64  0.81  0.98  1.26  1.58   1.82   
             2016-11-04  0.38  0.52  0.62  0.80  0.95  1.24  1.55   1.79   
             2016-11-07  0.41  0.54  0.63  0.82  0.99  1.29  1.60   1.83   

                         30 YR  
name  symbol date               
Alcoa AA     2016-11-01   2.58  
             2016-11-02   2.56  
             2016-11-03   2.60  
             2016-11-04   2.56  
             2016-11-07   2.60  

[5 rows x 26 columns]

In [18]:
### Calculated columns
#prices['EMA_20'] = prices.groupby(level=1).apply(lambda x: ta.EMA(x.close.values, 20))  # 20 day exponential moving average
prices  = test.copy().loc[(test.loc[:,'open':'30 YR'] != 0 ).all(axis=1)] # Don't touch this (:
#prices  = prices.copy().loc[(prices.loc[:,volume'] != 0 ).all(axis=1)] # Don't touch this (:

##price_close = prices.groupby(level=['symbol']).close

##prices['EMA_20'] = price_close.apply(lambda x: ta.EMA(x,20))   # 20 day exponential moving average
##prices['EMA_50'] = price_close.apply(lambda x: ta.EMA(x, 50))  # 50 day exponential moving average
##prices['EMA_200'] = price_close.apply(lambda x: ta.EMA(x, 200))  # 200 day exponential moving average
##prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
##prices['RSI'] = price_close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

#prices['CCI'] = prices.apply(lambda x: ta.CCI(high=x.high.va, low=x.low.values, close=x.close.values, timeperiod=20))  # 20 day commodity channel index

##prices['Momentum'] = price_close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum

#prices['Stoch_fastk'], prices['Stoch_fastd'] = prices.groupby(level=1).apply(lambda x: ta.STOCHF(high=x.high.values, low=x.low.values, close=x.close.values, fastk_period=14, fastd_period=3))  # Stochastic fast indicator
#prices['OBV'] = price_close.apply(lambda x: ta.OBV(np.asarray(x['close']), x['volume']))  # On Balance Volume
#prices['bbandupper'], prices['bbandmiddle'], prices['bbandlower'] =  prices.groupby(level=1).apply(lambda x: ta.BBANDS(x.close.values))


#prices['perc_change']=(prices['close']- prices['close'].shift(1))/prices['close'].shift(1)  # Getting the daily percentage change
#prices['5dayvol'] = prices['perc_change'].rolling(window=5, center=False).std()  # 5 day volatility 
#prices['5_day_target'] = prices.groupby(level=['symbol']).close.shift(-5) / prices.groupby(level=['symbol']).close  # target of X days
prices['5_day_target'] = prices.groupby(['name'])['close'].transform(lambda x:x.shift(-5))
prices['perc_change']=prices['5_day_target']/prices['close']
#prices['5dayvol'] = prices.groupby('symbol')['close']/prices.groupby(['symbol'])['close'].transform(lambda x:x.shift(1)).apply(pd.rolling_mean, 5, min_periods=5)  # 5 day volatility 

clean_prices = prices.loc[(prices['EMA_200'].notnull() & prices['5_day_target'].notnull())]


In [19]:
prices.head(100)

open     high      low    close      volume  \
name  symbol date                                                       
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0   
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0   
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0   
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0   
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0   
             2016-11-08  24.30  26.5900  24.3000  26.4000   6469979.0   
             2016-11-09  26.83  29.1400  26.0000  28.7200  10675856.0   
             2016-11-10  29.08  29.7000  28.3000  29.1000   8189887.0   
             2016-11-11  29.18  29.9900  27.6300  29.3000   5337239.0   
             2016-11-14  29.50  29.9400  28.6100  29.6200   5156737.0   
             2016-11-15  28.94  30.2300  28.8300  30.2100   3668883.0   
             2016-11-16  29.69  31.6900  29.6600  31.4200   6150098.0   
             2016-11-17  31.30  32.0200  30.7700  31.8500   4693203.0   
             2016-11-18  31.73  31.7700  30.1600  30.4200   4421730.0   
             2016-11-21  30.89  30.9800  29.5010  30.6900   2986107.0   
             2016-11-22  31.10  31.9100  30.5400  31.3400   4182721.0   
             2016-11-23  31.00  31.9300  30.0000  31.5100   4112265.0   
             2016-11-25  31.24  32.1900  31.1100  31.7000   3109628.0   
             2016-11-28  31.48  31.8000  30.7000  30.8600   5121935.0   
             2016-11-29  30.02  30.4999  29.7300  29.7300   4927708.0   
             2016-11-30  30.06  30.4400  28.8500  28.9700   5131539.0   
             2016-12-01  28.84  29.4200  28.6000  28.8800   5006186.0   
             2016-12-02  28.86  29.6250  28.7300  29.0400   3870676.0   
             2016-12-05  29.03  31.2768  28.8900  31.2200   7034136.0   
             2016-12-06  30.71  31.1900  30.2100  31.1500   3248595.0   
             2016-12-07  31.30  31.8900  30.7653  30.9000   5778259.0   
             2016-12-08  31.28  31.8400  30.8000  31.3100   3064978.0   
             2016-12-09  31.31  32.1100  31.0400  32.0500   4344295.0   
             2016-12-12  32.16  32.3500  30.8500  31.2400   3416012.0   
             2016-12-13  31.40  31.7300  30.0300  30.2400   4215143.0   
...                        ...      ...      ...      ...         ...   
             2017-02-13  38.79  39.7800  38.4500  38.4800   5366057.0   
             2017-02-14  38.57  39.2500  38.1800  38.5600   3754357.0   
             2017-02-15  38.25  38.3000  37.7300  37.8700  11488878.0   
             2017-02-16  37.82  37.8300  36.3500  36.4600   7040708.0   
             2017-02-17  36.24  36.6100  35.7900  36.2100   4516538.0   
             2017-02-21  36.31  37.0100  36.2300  36.7100   3393391.0   
             2017-02-22  36.40  36.7100  36.2100  36.6000   2587865.0   
             2017-02-23  36.86  36.8800  34.2600  34.3200   7139960.0   
             2017-02-24  33.81  34.6400  33.3500  34.4800   5763834.0   
             2017-02-27  34.52  35.3800  34.1900  34.7800   3898218.0   
             2017-02-28  34.73  35.3300  34.4000  34.5900   4034102.0   
             2017-03-01  35.93  38.1500  35.8500  37.9900  10564474.0   
             2017-03-02  37.64  38.1700  35.8300  35.8500   5098625.0   
             2017-03-03  36.06  36.8900  35.7500  36.3800   3238051.0   
             2017-03-06  36.80  37.4800  35.8000  36.9300   5764124.0   
             2017-03-07  36.81  36.8900  35.7500  35.8000   3753385.0   
             2017-03-08  35.90  36.8200  35.9000  35.9900   4485667.0   
             2017-03-09  36.07  36.5500  34.8900  35.3600   4826219.0   
             2017-03-10  35.90  36.0000  34.2350  34.5300   4052315.0   
             2017-03-13  35.21  35.5200  34.5300  34.7000   2491431.0   
             2017-03-14  34.45  34.5800  33.7700  34.1900   3113001.0   
             2017-03-15  34.40  35.2900  33.9500  35.1400   3753386.0   
           

In [21]:
clean_prices['perc_change'].min()

0.06716186939028164

In [22]:
test.head(500)

open     high      low    close  \
name                    symbol date                                           
Alcoa                   AA     2016-11-01  22.10  23.5500  21.7800  22.9219   
                               2016-11-02  22.67  23.2650  22.6358  22.8322   
                               2016-11-03  22.96  24.3493  22.9400  24.0680   
                               2016-11-04  23.82  25.5000  23.5600  25.1144   
                               2016-11-07  25.54  26.4100  24.8900  24.9948   
                               2016-11-08  24.30  26.5900  24.3000  26.4000   
                               2016-11-09  26.83  29.1400  26.0000  28.7200   
                               2016-11-10  29.08  29.7000  28.3000  29.1000   
                               2016-11-11  29.18  29.9900  27.6300  29.3000   
                               2016-11-14  29.50  29.9400  28.6100  29.6200   
                               2016-11-15  28.94  30.2300  28.8300  30.2100   
                               2016-11-16  29.69  31.6900  29.6600  31.4200   
                               2016-11-17  31.30  32.0200  30.7700  31.8500   
                               2016-11-18  31.73  31.7700  30.1600  30.4200   
                               2016-11-21  30.89  30.9800  29.5010  30.6900   
                               2016-11-22  31.10  31.9100  30.5400  31.3400   
                               2016-11-23  31.00  31.9300  30.0000  31.5100   
                               2016-11-25  31.24  32.1900  31.1100  31.7000   
                               2016-11-28  31.48  31.8000  30.7000  30.8600   
                               2016-11-29  30.02  30.4999  29.7300  29.7300   
                               2016-11-30  30.06  30.4400  28.8500  28.9700   
                               2016-12-01  28.84  29.4200  28.6000  28.8800   
                               2016-12-02  28.86  29.6250  28.7300  29.0400   
                               2016-12-05  29.03  31.2768  28.8900  31.2200   
                               2016-12-06  30.71  31.1900  30.2100  31.1500   
                               2016-12-07  31.30  31.8900  30.7653  30.9000   
                               2016-12-08  31.28  31.8400  30.8000  31.3100   
                               2016-12-09  31.31  32.1100  31.0400  32.0500   
                               2016-12-12  32.16  32.3500  30.8500  31.2400   
                               2016-12-13  31.40  31.7300  30.0300  30.2400   
...                                          ...      ...      ...      ...   
American Airlines Group AAL    2006-01-23  29.49  30.0500  29.1500  28.3274   
                               2006-01-24  29.90  31.4700  29.7500  29.8680   
                               2006-01-25  31.15  31.5100  30.7500  29.8487   
                               2006-01-26  31.05  31.2500  30.0000  29.1266   
                               2006-01-27  30.20  30.2000  28.3000  27.8171   
                               2006-01-30  28.73  29.5800  28.7300  27.9230   
                               2006-01-31  29.17  30.4100  29.0100  28.2504   
                               2006-02-01  29.10  29.6200  28.3500  28.2889   
                               2006-02-02  29.38  30.2000  29.0000  28.8955   
                               2006-02-03  30.01  30.0100  29.2000  28.4526   
                               2006-02-06  29.20  29.7000  28.6600  28.1830   
                               2006-02-07  29.35  29.4500  28.6200  27.9230   
                               2006-02-08  29.10  30.7600  29.0100  28.8763   
                               2006-02-09  29.95  30.4800  29.5500  29.2229   
                               2006-02-10  30.50  30.5000  29.7400  28.8859   
                               2006-02-13  31.05  32.1000  30.5500  30.0124   
                               2006-02-14  31.50  32.9000  31.4100  31.5819   
                               2006-02-15  32.55  34.2000  32.3500  32.4966   
               

In [23]:
list(clean_prices.columns.values)

['open',
 'high',
 'low',
 'close',
 'volume',
 'EMA_20',
 'EMA_50',
 'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 'bbandupper',
 'bbandmiddle',
 'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 '30 YR',
 '5_day_target',
 'perc_change']

In [24]:
testML=clean_prices.loc[:,[
 #'open',
 #'high',
 #'low',
 #'close',
 #'volume',
 #'EMA_20',
 #'EMA_50',
 #'EMA_200',
 'EMA_ratio',
 'RSI',
 'CCI',
 'Momentum',
 #'Stoch_fastk',
 #'Stoch_fastd',
 #'OBV',
 #'bbandupper',
 #'bbandmiddle',
 #'bbandlower',
 'bbandlowerii',
 'bbandupperii',
 '3 MO',
 '6 MO',
 '1 YR',
 '2 YR',
 '3 YR',
 '5 YR',
 '7 YR',
 '10 YR',
 #'30 YR',
'perc_change']]

In [25]:
testML[testML.isnull().any(axis=1)].head()

EMA_ratio        RSI         CCI  \
name                    symbol date                                           
Alcoa                   AA     2017-10-09   0.852329  60.232499 -119.271794   
American Airlines Group AAL    2006-10-09   0.889025  55.312252 -108.737480   
                               2007-10-08   1.225757  57.119257   97.935045   
                               2007-11-12   1.250296  32.446514  -16.840325   
                               2008-10-13   1.572072  46.573720   81.809532   

                                           Momentum  bbandlowerii  \
name                    symbol date                                 
Alcoa                   AA     2017-10-09    1.7800      0.995675   
American Airlines Group AAL    2006-10-09   -0.6163      0.997060   
                               2007-10-08    4.4195      0.922231   
                               2007-11-12   -4.8528      0.943504   
                               2008-10-13    0.0771      0.562015   

                                           bbandupperii  3 MO  6 MO  1 YR  \
name                    symbol date                                         
Alcoa                   AA     2017-10-09      1.035598   NaN   NaN   NaN   
American Airlines Group AAL    2006-10-09      1.027266   NaN   NaN   NaN   
                               2007-10-08      1.011429   NaN   NaN   NaN   
                               2007-11-12      1.067933   NaN   NaN   NaN   
                               2008-10-13      1.068257   NaN   NaN   NaN   

                                           2 YR  3 YR  5 YR  7 YR  10 YR  \
name                    symbol date                                        
Alcoa                   AA     2017-10-09   NaN   NaN   NaN   NaN    NaN   
American Airlines Group AAL    2006-10-09   NaN   NaN   NaN   NaN    NaN   
                               2007-10-08   NaN   NaN   NaN   NaN    NaN   
                               2007-11-12   NaN   NaN   NaN   NaN    NaN   
                               2008-10-13   NaN   NaN   NaN   NaN    NaN   

                                           perc_change  
name                    symbol date                     
Alcoa                   AA     2017-10-09     1.024857  
American Airlines Group AAL    2006-10-09     0.958325  
                               2007-10-08     0.922472  
                               2007-11-12     0.935541  
                               2008-10-13     1.301055

In [26]:
testML=testML.fillna(method='pad')

In [27]:
prices.loc[:,['close','5_day_target','perc_change']].sort_values(['perc_change'], ascending=True).head(100)

close  \
name                                symbol date                    
Ally Financial                      ALLY   2014-03-28  8344.0773   
                                           2014-04-02  8344.0773   
                                           2014-03-27  8175.2665   
                                           2014-03-10  8175.2665   
                                           2014-02-06  7813.5290   
Markel                              MKL    2010-04-14   387.7200   
                                           2010-04-15   386.4000   
                                           2010-04-13   377.0200   
                                           2010-04-16   380.7500   
                                           2010-04-12   376.9000   
Freddie Mac                         FMCC   2008-09-05     5.1000   
Fannie Mae                          FNMA   2008-09-03     7.3200   
                                           2008-09-05     7.0400   
                                           2008-08-29     6.8400   
AIG                                 AIG    2008-09-10   272.2593   
Freddie Mac                         FMCC   2008-09-04     4.9500   
Fannie Mae                          FNMA   2008-09-04     6.4200   
Freddie Mac                         FMCC   2008-09-03     5.3800   
Fannie Mae                          FNMA   2008-09-02     7.4300   
Williams                            WMB    2002-07-18     3.2298   
AIG                                 AIG    2008-09-11   273.0371   
Freddie Mac                         FMCC   2008-09-02     5.1800   
WellCare Health Plans               WCG    2007-10-23   122.2700   
Genworth Financial                  GNW    2008-11-05     5.1800   
Freddie Mac                         FMCC   2008-08-29     4.5100   
Williams                            WMB    2002-07-16     3.0305   
                                           2002-07-17     3.1684   
AIG                                 AIG    2008-09-09   285.7944   
Williams                            WMB    2002-07-19     2.6370   
AIG                                 AIG    2008-09-08   354.0926   
...                                                          ...   
Hartford Financial Services Group   HIG    2008-11-14    10.3750   
Builders FirstSource                BLDR   2008-11-14     2.5500   
Freddie Mac                         FMCC   2010-06-14     1.2000   
Citigroup                           C      2008-11-14    91.2957   
Auto-Owners Insurance               SEE    2002-07-29    14.9743   
Sealed Air                          SEE    2002-07-29    14.9743   
Freddie Mac                         FMCC   2010-06-15     1.2200   
Las Vegas Sands                     LVS    2008-10-21     9.2212   
                                           2008-11-04     9.9259   
Builders FirstSource                BLDR   2008-11-17     2.5000   
Genesis Healthcare                  GEN    2010-07-06     6.2200   
Fannie Mae                          FNMA   2008-07-08    17.6200   
WellCare Health Plans               WCG    2008-11-07    23.2700   
                                           2008-11-12    19.8700   
Morgan Stanley                      MS     2008-10-03    20.6919   
Massachusetts Mutual Life Insurance MS     2008-10-03    20.6919   
HP                                  HPQ    2015-10-16    12.8825   
Xcel Energy                         XEL    2002-07-22     7.1286   
iHeartMedia                         IHRT   2017-11-02     1.3500   
Freddie Mac                         FMCC   2008-09-10     0.6600   
Genworth Financial                  GNW    2008-09-22    11.9654   
HP                                  HPQ    2015-10-12    13.0789   
Priceline Group                     PCLN   2001-09-06    29.7000   
AES                                 AES    2002-02-13     7.5259   
Auto-Owners Insurance               SEE    2002-07-24    14.0030   
Sealed Air                          SEE    2002-07-24    14.0030   
HP                                  HPQ    2015

In [28]:
    #x_train, x_test, y_train, y_test = train_test_split(clean_prices.loc[:,'EMA_ratio':'bbandlower'], clean_prices.loc[:,['7_day_target']], shuffle=False, test_size=90)
    test_size = 3
    # Get columns
    X = testML.copy().loc[:,'EMA_ratio':'10 YR'] 
    y = testML.copy().loc[:, ['perc_change']]

In [29]:
    # Get series of ingroup rownumbers
    X['num']= X.groupby(level=1).cumcount(ascending=False)
    
    testML.loc[ testML.EMA_ratio == testML.EMA_ratio.max() ] 

,,,EMA_ratio,RSI,CCI,Momentum,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,perc_change
name,symbol,date,,,,,,,,,,,,,,,
Ally Financial,ALLY,2015-01-09,15.947293,59.149339,104.141459,0.0096,0.954267,1.005365,0.02,0.08,0.22,0.59,0.96,1.45,1.75,1.98,0.854272


In [30]:
    # Filter
    X_train = X.loc[X.num > test_size ].drop(columns='num')
    X_test =  X.loc[X.num <= test_size ].drop(columns='num')
    y_train = y.loc[X.num > test_size ]
    y_test =  y.loc[X.num <= test_size ]
    
    X_test.head(25)

EMA_ratio        RSI         CCI  \
name                    symbol date                                           
Alcoa                   AA     2018-05-09   0.905885  53.575730  -47.314121   
                               2018-05-10   0.905565  54.575739  -67.447046   
                               2018-05-11   0.905216  55.165108  -20.622896   
                               2018-05-14   0.905732  48.647566 -166.666667   
Nationwide              AAL    2018-05-09   1.031121  32.640112  -80.621057   
                               2018-05-10   1.034387  35.689375  -86.965629   
                               2018-05-11   1.037145  38.784200   -2.036809   
                               2018-05-14   1.039867  38.288131   30.034862   
Advance Auto Parts      AAP    2018-05-09   0.995096  61.789178  -36.405591   
                               2018-05-10   0.993684  62.236152  106.686332   
                               2018-05-11   0.991910  65.346529  144.139923   
                               2018-05-14   0.989810  68.109071  103.594663   
Apple                   AAPL   2018-05-09   0.951683  70.033809   70.246199   
                               2018-05-10   0.949489  72.319115  135.206287   
                               2018-05-11   0.947550  70.757887   74.455895   
                               2018-05-14   0.945810  69.762861   43.594184   
AbbVie                  ABBV   2018-05-09   0.940073  58.227915  146.430579   
                               2018-05-10   0.939752  61.481561  158.926149   
                               2018-05-11   0.939114  63.893621   99.945581   
                               2018-05-14   0.938075  66.774469  107.817273   
AmerisourceBergen       ABC    2018-05-09   0.988789  40.336193  -70.648695   
                               2018-05-10   0.990311  42.564070  106.268649   
                               2018-05-11   0.991331  46.171719   84.934124   
                               2018-05-14   0.992178  47.229315  114.888017   
Asbury Automotive Group ABG    2018-05-09   0.955159  55.028553  -34.471853   

                                           Momentum  bbandlowerii  \
name                    symbol date                                 
Alcoa                   AA     2018-05-09    1.5400      0.988974   
                               2018-05-10    0.8500      0.988806   
                               2018-05-11    2.3600      0.985965   
                               2018-05-14    1.1300      0.999250   
Nationwide              AAL    2018-05-09   -3.6436      0.994471   
                               2018-05-10   -0.2304      0.982000   
                               2018-05-11   -0.6980      0.969340   
                               2018-05-14   -0.3691      0.972582   
Advance Auto Parts      AAP    2018-05-09   10.1700      0.973973   
                               2018-05-10    5.0500      0.977376   
                               2018-05-11    3.6000      0.960514   
                               2018-05-14    7.1600      0.950202   
Apple                   AAPL   2018-05-09   23.6186      0.942140   
                               2018-05-10   25.7204      0.959035   
                               2018-05-11   26.8959      0.971176   
                               2018-05-14   23.5272      0.981865   
AbbVie                  ABBV   2018-05-09    9.3100      0.977427   
                               2018-05-10    5.4100      0.954044   
                               2018-05-11    5.4500      0.938377   
                               2018-05-14    9.3400      0.927146   
AmerisourceBergen       ABC    2018-05-09   -6.6801      0.989832   
                               2018-05-10   -6.5208      0.980531   
                               2018-05-11   -6.1624      0.962223   
                               2018-05-14   -3.2853      0.958537   
Asbury Automotive Group ABG    2018-05-09   -0.6500      0.985813   

                                           bb

In [31]:
y_train.min()

perc_change    0.067162
dtype: float64

In [32]:
y_train.head()

perc_change
name  symbol date                   
Alcoa AA     2017-08-17     1.093651
             2017-08-18     1.091367
             2017-08-21     1.096179
             2017-08-22     1.081217
             2017-08-23     1.049440

In [33]:
y_train.sort_values(['perc_change'], axis=1)

KeyError: 'perc_change'

In [34]:
X_train.head(20)

EMA_ratio        RSI         CCI  Momentum  \
name  symbol date                                                     
Alcoa AA     2017-08-17   0.930952  55.546179   37.966428      0.71   
             2017-08-18   0.930128  55.292702  -29.459415      0.34   
             2017-08-21   0.928852  58.879643   43.007073      0.48   
             2017-08-22   0.926723  64.496425  130.159556      0.19   
             2017-08-23   0.923809  69.018740  122.125631      2.44   
             2017-08-24   0.920899  69.914514   96.775456      3.56   
             2017-08-25   0.918287  68.918184   72.790217      3.43   
             2017-08-28   0.915188  72.082142   98.236776      4.98   
             2017-08-29   0.911732  74.499112  125.363725      5.50   
             2017-08-30   0.908465  74.754338  122.811457      3.86   
             2017-08-31   0.904900  76.936396   95.564907      6.08   
             2017-09-01   0.900818  79.709621  120.709145      7.28   
             2017-09-05   0.897449  74.549931   64.384952      5.95   
             2017-09-06   0.894327  74.578514    0.510512      4.66   
             2017-09-07   0.891247  75.468628   49.227546      3.67   
             2017-09-08   0.889763  59.589874 -152.054795      1.27   
             2017-09-11   0.887778  63.456596  -34.198113      2.39   
             2017-09-12   0.885071  68.104524  100.477653      2.83   
             2017-09-13   0.882633  67.510070   61.167228      1.91   
             2017-09-14   0.880461  66.743266   34.651361      1.71   

                         bbandlowerii  bbandupperii  3 MO  6 MO  1 YR  2 YR  \
name  symbol date                                                             
Alcoa AA     2017-08-17      0.965553      1.039738  1.00  1.11  1.24  1.32   
             2017-08-18      0.966431      1.040773  1.02  1.13  1.24  1.33   
             2017-08-21      0.959135      1.024333  1.00  1.11  1.23  1.32   
             2017-08-22      0.930955      1.010509  1.00  1.13  1.24  1.33   
             2017-08-23      0.887185      1.011110  1.00  1.11  1.22  1.32   
             2017-08-24      0.892050      1.027640  1.02  1.11  1.23  1.33   
             2017-08-25      0.926044      1.033189  1.03  1.11  1.23  1.35   
             2017-08-28      0.938221      1.011506  0.98  1.12  1.24  1.33   
             2017-08-29      0.937052      1.005087  1.03  1.13  1.23  1.33   
             2017-08-30      0.941650      1.015278  1.03  1.11  1.23  1.33   
             2017-08-31      0.930996      1.013854  1.01  1.08  1.23  1.33   
             2017-09-01      0.921415      1.007359  1.02  1.10  1.24  1.35   
             2017-09-05      0.952896      1.023060  1.03  1.13  1.23  1.30   
             2017-09-06      0.964761      1.023625  1.07  1.17  1.24  1.30   
             2017-09-07      0.977654      1.012062  1.05  1.15  1.21  1.27   
             2017-09-08      0.998538      1.078252  1.04  1.14  1.22  1.27   
             2017-09-11      0.972813      1.043609  1.05  1.16  1.24  1.32   
             2017-09-12      0.940766      1.018079  1.03  1.16  1.27  1.33   
             2017-09-13      0.942183      1.024863  1.04  1.16  1.27  1.35   
             2017-09-14      0.944415      1.028085  1.05  1.17  1.28  1.37   

                         3 YR  5 YR  7 YR  10 YR  
name  symbol date                                 
Alcoa AA     2017-08-17  1.46  1.76  2.01   2.19  
             2017-08-18  1.47  1.77  2.01   2.19  
             2017-08-21  1.46  1.76  2.00   2.18  
             2017-08-22  1.48  1.80  2.04   2.22  
             2017-08-23  1.45  1.76  1.99   2.17  
             2017-08-24  1.47  1.78  2.01   2.19  
             2017-08-25  1.47  1.77  2.00   2.17  
             2017-08-28  1.46  1.74  1.99   2.16  
             2017-08-29  1.43  1.70  1.96   2.13  
             2017-08-30  1.44  1.72  1.97   2.15  
             2017-08-31  1.44  1.70  1.95   2.12  
             2017-09-01  1.46  1.73  1.99   2.16  
         

In [35]:
    %timeit
    forest = RandomForestRegressor()
    forest.fit(X_train,y_train)
    y_pred = forest.predict(X_test)
    y_pred = pd.DataFrame({'ans': y_pred})
    

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
test.head()

open     high      low    close      volume  EMA_20  \
name  symbol date                                                               
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0     NaN   
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0     NaN   
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0     NaN   
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0     NaN   
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0     NaN   

                         EMA_50  EMA_200  EMA_ratio  RSI  ...    bbandupperii  \
name  symbol date                                         ...                   
Alcoa AA     2016-11-01     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-02     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-03     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-04     NaN      NaN        NaN  NaN  ...             NaN   
             2016-11-07     NaN      NaN        NaN  NaN  ...        1.037733   

                         3 MO  6 MO  1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  \
name  symbol date                                                          
Alcoa AA     2016-11-01  0.35  0.50  0.65  0.83  0.99  1.30  1.61   1.83   
             2016-11-02  0.37  0.51  0.64  0.81  0.98  1.26  1.57   1.81   
             2016-11-03  0.38  0.52  0.64  0.81  0.98  1.26  1.58   1.82   
             2016-11-04  0.38  0.52  0.62  0.80  0.95  1.24  1.55   1.79   
             2016-11-07  0.41  0.54  0.63  0.82  0.99  1.29  1.60   1.83   

                         30 YR  
name  symbol date               
Alcoa AA     2016-11-01   2.58  
             2016-11-02   2.56  
             2016-11-03   2.60  
             2016-11-04   2.56  
             2016-11-07   2.60  

[5 rows x 26 columns]

In [37]:
#final = pd.DataFrame({'close':test.tail(90).reset_index().close, 'ratio':y_pred['ans']})
#final['pred_price']= final['close'] * final['ratio']

In [38]:
testML.head()

EMA_ratio        RSI         CCI  Momentum  \
name  symbol date                                                     
Alcoa AA     2017-08-17   0.930952  55.546179   37.966428      0.71   
             2017-08-18   0.930128  55.292702  -29.459415      0.34   
             2017-08-21   0.928852  58.879643   43.007073      0.48   
             2017-08-22   0.926723  64.496425  130.159556      0.19   
             2017-08-23   0.923809  69.018740  122.125631      2.44   

                         bbandlowerii  bbandupperii  3 MO  6 MO  1 YR  2 YR  \
name  symbol date                                                             
Alcoa AA     2017-08-17      0.965553      1.039738  1.00  1.11  1.24  1.32   
             2017-08-18      0.966431      1.040773  1.02  1.13  1.24  1.33   
             2017-08-21      0.959135      1.024333  1.00  1.11  1.23  1.32   
             2017-08-22      0.930955      1.010509  1.00  1.13  1.24  1.33   
             2017-08-23      0.887185      1.011110  1.00  1.11  1.22  1.32   

                         3 YR  5 YR  7 YR  10 YR  perc_change  
name  symbol date                                              
Alcoa AA     2017-08-17  1.46  1.76  2.01   2.19     1.093651  
             2017-08-18  1.47  1.77  2.01   2.19     1.091367  
             2017-08-21  1.46  1.76  2.00   2.18     1.096179  
             2017-08-22  1.48  1.80  2.04   2.22     1.081217  
             2017-08-23  1.45  1.76  1.99   2.17     1.049440

In [39]:
ml=test.sort_index(level=1, ascending=True)
ml=test.groupby(['name']).tail(1).loc[:,'EMA_ratio':'10 YR']  #tail to get the last date entry for each day

In [40]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
name,symbol,date,,,,,,,,,,,,,,,,,
Alcoa,AA,2018-05-21,0.912533,42.723947,-5.838641,-4.180,50.987286,50.528,50.068714,0.994019,1.012255,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
American Airlines Group,AAL,2018-05-21,1.047790,48.865831,83.429942,1.840,44.524372,43.708,42.891628,0.968646,1.005519,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
Nationwide,AAL,2018-05-21,1.047790,48.865831,83.429942,1.840,44.524372,43.708,42.891628,0.968646,1.005519,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
Advance Auto Parts,AAP,2018-05-21,0.983659,58.788669,-107.678799,3.010,120.395596,119.242,118.088404,0.991090,1.010454,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06
Apple,AAPL,2018-05-21,0.939472,65.274117,91.825571,3.184,188.529465,187.110,185.690535,0.989663,1.004794,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06


In [41]:
ml[ml.isnull().any(axis=1)].head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandupper,bbandmiddle,bbandlower,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
name,symbol,date,,,,,,,,,,,,,,,,,
Berkshire Hathaway,BRKA,2018-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06


In [42]:
ml=ml.loc[ml['EMA_ratio'].notnull()]

In [43]:
ml.drop(['bbandupper','bbandmiddle','bbandlower'], axis=1, inplace=True)

In [44]:
ml['5dayforecast'] = forest.predict(ml)

In [45]:
ml.head()

,,,EMA_ratio,RSI,CCI,Momentum,bbandlowerii,bbandupperii,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,5dayforecast
name,symbol,date,,,,,,,,,,,,,,,
Alcoa,AA,2018-05-21,0.912533,42.723947,-5.838641,-4.180,0.994019,1.012255,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.005992
American Airlines Group,AAL,2018-05-21,1.047790,48.865831,83.429942,1.840,0.968646,1.005519,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.007671
Nationwide,AAL,2018-05-21,1.047790,48.865831,83.429942,1.840,0.968646,1.005519,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.007671
Advance Auto Parts,AAP,2018-05-21,0.983659,58.788669,-107.678799,3.010,0.991090,1.010454,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.015251
Apple,AAPL,2018-05-21,0.939472,65.274117,91.825571,3.184,0.989663,1.004794,1.93,2.14,2.35,2.58,2.73,2.9,3.02,3.06,1.018321


In [46]:
prices.head()

open     high      low    close      volume  EMA_20  \
name  symbol date                                                               
Alcoa AA     2016-11-01  22.10  23.5500  21.7800  22.9219  32216510.0     NaN   
             2016-11-02  22.67  23.2650  22.6358  22.8322  10755328.0     NaN   
             2016-11-03  22.96  24.3493  22.9400  24.0680   8083097.0     NaN   
             2016-11-04  23.82  25.5000  23.5600  25.1144   6878500.0     NaN   
             2016-11-07  25.54  26.4100  24.8900  24.9948   8377025.0     NaN   

                         EMA_50  EMA_200  EMA_ratio  RSI     ...       6 MO  \
name  symbol date                                            ...              
Alcoa AA     2016-11-01     NaN      NaN        NaN  NaN     ...       0.50   
             2016-11-02     NaN      NaN        NaN  NaN     ...       0.51   
             2016-11-03     NaN      NaN        NaN  NaN     ...       0.52   
             2016-11-04     NaN      NaN        NaN  NaN     ...       0.52   
             2016-11-07     NaN      NaN        NaN  NaN     ...       0.54   

                         1 YR  2 YR  3 YR  5 YR  7 YR  10 YR  30 YR  \
name  symbol date                                                     
Alcoa AA     2016-11-01  0.65  0.83  0.99  1.30  1.61   1.83   2.58   
             2016-11-02  0.64  0.81  0.98  1.26  1.57   1.81   2.56   
             2016-11-03  0.64  0.81  0.98  1.26  1.58   1.82   2.60   
             2016-11-04  0.62  0.80  0.95  1.24  1.55   1.79   2.56   
             2016-11-07  0.63  0.82  0.99  1.29  1.60   1.83   2.60   

                         5_day_target  perc_change  
name  symbol date                                   
Alcoa AA     2016-11-01         26.40     1.151737  
             2016-11-02         28.72     1.257873  
             2016-11-03         29.10     1.209074  
             2016-11-04         29.30     1.166661  
             2016-11-07         29.62     1.185046  

[5 rows x 28 columns]

In [47]:
result=ml.join(prices, lsuffix='l',how ='left')

In [48]:
result[['close','5dayforecast']]

,,,close,5dayforecast
name,symbol,date,,
Alcoa,AA,2018-05-21,50.37,1.005992
American Airlines Group,AAL,2018-05-21,44.28,1.007671
Nationwide,AAL,2018-05-21,44.28,1.007671
Advance Auto Parts,AAP,2018-05-21,119.15,1.015251
Apple,AAPL,2018-05-21,187.63,1.018321
AbbVie,ABBV,2018-05-21,106.23,1.019458
AmerisourceBergen,ABC,2018-05-21,84.34,1.003321
Asbury Automotive Group,ABG,2018-05-21,70.80,1.008849
ABM Industries,ABM,2018-05-21,30.02,0.987200


In [49]:
result=result[result.close.notnull()]

In [50]:
result=result[['close','5dayforecast']]

In [51]:
result.head()

,,,close,5dayforecast
name,symbol,date,,
Alcoa,AA,2018-05-21,50.37,1.005992
American Airlines Group,AAL,2018-05-21,44.28,1.007671
Nationwide,AAL,2018-05-21,44.28,1.007671
Advance Auto Parts,AAP,2018-05-21,119.15,1.015251
Apple,AAPL,2018-05-21,187.63,1.018321


In [52]:
result['5day_price']=result['close']*result['5dayforecast']

In [53]:
result['5day%_change']=(result['5dayforecast']-1)*100

In [54]:
result

,,,close,5dayforecast,5day_price,5day%_change
name,symbol,date,,,,
Alcoa,AA,2018-05-21,50.37,1.005992,50.671816,0.599199
American Airlines Group,AAL,2018-05-21,44.28,1.007671,44.619682,0.767123
Nationwide,AAL,2018-05-21,44.28,1.007671,44.619682,0.767123
Advance Auto Parts,AAP,2018-05-21,119.15,1.015251,120.967181,1.525120
Apple,AAPL,2018-05-21,187.63,1.018321,191.067491,1.832058
AbbVie,ABBV,2018-05-21,106.23,1.019458,108.297000,1.945778
AmerisourceBergen,ABC,2018-05-21,84.34,1.003321,84.620120,0.332132
Asbury Automotive Group,ABG,2018-05-21,70.80,1.008849,71.426488,0.884870
ABM Industries,ABM,2018-05-21,30.02,0.987200,29.635758,-1.279954


In [55]:
prep = result[['close','5day_price','5day%_change']]
prep.sort_values("5day%_change",ascending=False, inplace=True)
prep.reset_index(inplace=True)
prep.set_index('name')
tdf=timedelta(days=5)
prep=prep[prep['date']>=(datetime.now()-tdf)]

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
prep.shape

(481, 6)

In [57]:
prep.tail(20)

,name,symbol,date,close,5day_price,5day%_change
473,NRG Energy,NRG,2018-05-21,33.50,32.762347,-2.201950
474,Kraft Heinz,KHC,2018-05-21,56.45,55.201985,-2.210833
475,Xerox,XRX,2018-05-21,27.98,27.340695,-2.284865
477,Bed Bath & Beyond,BBBY,2018-05-21,18.01,17.566238,-2.463973
479,Conagra Brands,CAG,2018-05-21,36.91,35.941585,-2.623721
480,Celgene,CELG,2018-05-21,74.69,72.656257,-2.722911
482,General Electric,GE,2018-05-21,15.26,14.774313,-3.182746
483,Frontier Communications,FTR,2018-05-21,9.60,9.271939,-3.417298
484,Newell Brands,NWL,2018-05-21,26.39,25.468820,-3.490642
485,Land O’Lakes,NWL,2018-05-21,26.39,25.468820,-3.490642


In [58]:
prep.head(20)

,name,symbol,date,close,5day_price,5day%_change
0,Farmers Insurance Exchange,JCP,2018-05-21,2.500,2.750157,10.006291
1,J.C. Penney,JCP,2018-05-21,2.500,2.750157,10.006291
2,Campbell Soup,CPB,2018-05-21,33.580,36.668912,9.198665
3,Williams,WMB,2018-05-21,27.620,29.075611,5.270135
4,INTL FCStone,INTL,2018-05-21,48.150,50.398091,4.668932
5,Sanmina,SANM,2018-05-21,30.025,31.423857,4.658975
6,Ascena Retail Group,ASNA,2018-05-21,3.140,3.271310,4.181856
7,CBS,CBS,2018-05-21,52.000,54.143018,4.121189
8,Lincoln National,LNC,2018-05-21,69.830,72.578479,3.935958
9,Sonic Automotive,SAH,2018-05-21,22.000,22.852135,3.873340
